## Parametry klasy Pacjent:
- **id**: identyfikator pacjenta
- **arrival_time**: czas przybycia pacjenta do kliniki
- **service_start_time**: czas rozpoczęcia wizyty
- **service_end_time**: czas zakonczenia wizyty
- **room**: gabinet przypisany pacjentowi

In [1]:
class Pacjent:
    def __init__(self, id):
        self.id = id
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None
        self.room = None

## Parametry klasy Gabinet:
- **id**: identyfikator gabinetu
- **env**: środowiko symulacji
- **patients_served**: ilość obsłużonych pacjentów
- **queue**: kolejka do gabinetu
- **resource**: wskazuje na obiekt z biblioteki sympy który odpowiada za faktyczne obsługiwanie symulacji

In [ ]:
class Gabinet:
    def __init__(self, id, env):
        self.id = id
        self.env = env
        self.patients_served = 0
        self.no_show = 0
        self.queue = []
        self.resource = simpy.Resource(env, capacity=1)

## Parametry klasy Clinic:
- **curr_patient_id**: aktualny identyfikator pacjenta u żywany przy tworzeniu pacjentów
- **env**: środowsko symuacji
- **service_time**: czas obsługi jednego pacjanta
- **no_show**: prawdopodobieństwo że pacjent nie zjawi na wizytę
- **seed**: seed generatora liczby pseudolosowych
- **list_rooms**: lista gabinetów
- **processed_patients**: lista, która przechowuje informacje o pacjkentach którzy już zostali obsłużeni
- **sim_time**: całkowity czas symulacji

In [ ]:
class Clinic:
    def __init__(self, env, number_of_rooms, service_time, no_show=0.2, seed=None, sim_time=480):
        self.curr_patient_id = 1
        self.env = env
        self.service_time = service_time
        self.no_show = no_show
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1, env=self.env) for i in range(number_of_rooms)]
        self.processed_patients = []
        self.sim_time = sim_time

        if self.seed:
            random.seed(self.seed)

In [ ]:
    def czas(self):
        hours = 8 + self.env.now//60
        minutes = self.env.now%60
        if len(str(minutes))==1:
            return f"{hours}:0{minutes}"
        return f"{hours}:{minutes}"

In [ ]:
    def generate_patients(self, room):
        def time_between_new_patients():
            return self.service_time

        while True:
            if self.env.now > self.sim_time - self.service_time:  # Pacjenci nie przychodzą przed {service_time} zamknięciem
                break
            patient = Pacjent(id=f"{room.id}.{self.curr_patient_id}")
            patient.room = room.id
            if random.random() < self.no_show: #pacjent nie  przyszedl
                print(f'Czas {self.czas()}: Pacjent {patient.id} nie pojawił w gabinecie {room.id} w ciągu 15 minut')
                room.no_show += 1

            else: #pacjent przyszedl
                patient.arrival_time = self.env.now
                print(f"Czas {self.czas()}: Pacjent {patient.id} przybył do kliniki")
                self.env.process(self.serve_patient(patient, room))

            self.curr_patient_id += 1
            yield self.env.timeout(time_between_new_patients())

In [ ]:
    def serve_patient(self, patient, room):
        room.queue.append(patient.id)
        with room.resource.request() as request:
            yield request
            patient.service_start_time = self.env.now
            room.queue.remove(patient.id)
            print(f"Czas {self.czas()}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
            yield self.env.timeout(self.service_time)
            patient.service_end_time = self.env.now
            room.patients_served += 1
            print(f"Czas {self.czas()}: Pacjent {patient.id} wychodzi z gabinetu {room.id}")
            self.processed_patients.append(patient)

Metoda **run** odpowiada za uruchomienie symulacji po wywołaniu dla każdego gabinetu rozpoczyna osobny generator pacjentów.

Czas trwania jest wydłużony o 0.01 aby komunikaty które pojawiały się w ostatniej "minucie" symulacji mogły się wyświetlić.

In [ ]:
    def run(self):
        for room in self.list_rooms:
            self.env.process(self.generate_patients(room))
        env.run(until=self.sim_time + 0.01)

## Pełny kod gotowy do uruchomienia:

In [ ]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import simpy
import seaborn
import matplotlib.pyplot as plt
import random


class Pacjent:
    def __init__(self, id):
        self.id = id
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None
        self.room = None


class Gabinet:
    def __init__(self, id, env):
        self.id = id
        self.env = env
        self.patients_served = 0
        self.no_show = 0
        self.queue = []
        self.resource = simpy.Resource(env, capacity=1)


class Clinic:
    def __init__(self, env, number_of_rooms, service_time, no_show=0.2, seed=None, sim_time=480):
        self.curr_patient_id = 1
        self.env = env
        self.service_time = service_time
        self.no_show = no_show
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1, env=self.env) for i in range(number_of_rooms)]
        self.processed_patients = []
        self.sim_time = sim_time

        if self.seed:
            random.seed(self.seed)

    def czas(self):
        hours = 8 + self.env.now//60
        minutes = self.env.now%60
        if len(str(minutes))==1:
            return f"{hours}:0{minutes}"
        return f"{hours}:{minutes}"

    def generate_patients(self, room):
        def time_between_new_patients():
            return self.service_time

        while True:
            if self.env.now > self.sim_time - self.service_time:  # Pacjenci nie przychodzą przed {service_time} zamknięciem
                break
            patient = Pacjent(id=f"{room.id}.{self.curr_patient_id}")
            patient.room = room.id
            if random.random() < self.no_show: #pacjent nie  przyszedl
                print(f'Czas {self.czas()}: Pacjent {patient.id} nie pojawił w gabinecie {room.id} w ciągu 15 minut')
                room.no_show += 1

            else: #pacjent przyszedl
                patient.arrival_time = self.env.now
                print(f"Czas {self.czas()}: Pacjent {patient.id} przybył do kliniki")
                self.env.process(self.serve_patient(patient, room))

            self.curr_patient_id += 1
            yield self.env.timeout(time_between_new_patients())

    def serve_patient(self, patient, room):
        room.queue.append(patient.id)
        with room.resource.request() as request:
            yield request
            patient.service_start_time = self.env.now
            room.queue.remove(patient.id)
            print(f"Czas {self.czas()}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
            yield self.env.timeout(self.service_time)
            patient.service_end_time = self.env.now
            room.patients_served += 1
            print(f"Czas {self.czas()}: Pacjent {patient.id} wychodzi z gabinetu {room.id}")
            self.processed_patients.append(patient)

    def run(self):
        for room in self.list_rooms:
            self.env.process(self.generate_patients(room))
        env.run(until=self.sim_time + 0.01)

    def stats(self):
        def patient_bar_plot():
            patients_served_ls = []
            id_ls = []
            for room in self.list_rooms:
                patients_served_ls.append(room.patients_served)
                id_ls.append(str(room.id))
            fig = seaborn.barplot(x=id_ls, y=patients_served_ls)
            fig.set_xlabel('Gabinet')
            fig.set_ylabel('Ilosc Pacjentow')
            plt.savefig("patients_served_noshow.png")
            plt.show()

        patient_bar_plot()

#z umowieniami
env = simpy.Environment()

clinic = Clinic(env, number_of_rooms=3, service_time=15)

clinic.run()

clinic.stats()